In [1]:
# importing all the necassary libraries
from langchain_community.document_loaders.firecrawl import FireCrawlLoader
from dotenv import load_dotenv
import os
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import DirectoryLoader

In [2]:
# loading environment file
load_dotenv(override=True)
Fire_Crawl_Api_Key = os.getenv('FIRECRAWL_API_KEY')
Huggingface_api_key = os.getenv('HUGGINGFACE_API_KEY')

In [3]:
# loading markdown files
loader = DirectoryLoader("markdown", glob="**/*.md", loader_cls=TextLoader)
docs = loader.load()

In [4]:
# converting docs into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=700, chunk_overlap=100)
chunks = text_splitter.split_documents(docs)

In [5]:
# intializing embedding model
embeddor = HuggingFaceInferenceAPIEmbeddings(api_key=Huggingface_api_key, model_name="sentence-transformers/all-MiniLM-L6-v2")


# creating vector database
vector_database = Chroma.from_documents(documents=chunks, embedding=embeddor, persist_directory='Streamlit_db')


In [6]:

from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
import gradio as gr

# create a new Chat with OpenAI
llm = ChatOpenAI(temperature=0.7, model_name='gpt-4o-mini')

# set up the conversation memory for the chat
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# the retriever is an abstraction over the VectorStore that will be used during RAG; k is how many chunks to use
retriever = vector_database.as_retriever()

# putting it together: set up the conversation chain with the GPT 3.5 LLM, the vector store and memory
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

def chat(question, history):
    result = conversation_chain.invoke({"question": question})
    return result["answer"]
view = gr.ChatInterface(chat, type="messages").launch(inbrowser=True)


C:\Users\pandh\AppData\Local\Temp\ipykernel_12800\1980539541.py:9: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
